# Assignment4: Sentiment Analysis

Use the UCI Sentiment Labelled Sentences Data Set in this assignment:  https://archive.ics.uci.edu/ml/datasets/Sentiment+Labelled+Sentences.
Create a sentiment analyzer using the TfidfVectorizer representation of the amazon reviews from the UCI Sentiment Labelled Sentences Data Set. Use the TfidfVectorizer function in Sklearn and train both a logistic regression model.

Load the dataset and select the amazon reviews

In [1]:
import pandas as pd
import numpy as np
import sys

directory = sys.path[0]
# amazon
amazon = open('amazon_cells_labelled.txt').read()


a_labels, a_texts = [], []
for i, line in enumerate(amazon.split('\n')):
    content = line.split('\t')
    if len(content) > 1:
        a_texts.append(content[0])
        a_labels.append(content[1])

df_a = pd.DataFrame()
df_a['label'] = a_labels
df_a['text'] = a_texts
df_a['source'] = 'amazon'

for i, line in enumerate(amazon.split('\n')):
    content = line.split('\t')
    if len(content) > 1:
        a_texts.append(content[0])
        a_labels.append(content[1])
        
df = pd.DataFrame()
df['label'] = a_labels
df['text'] = a_texts

df.label = df.label.astype(int)
df.info()

display(df_a.head())
display(df_a.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
label    2000 non-null int64
text     2000 non-null object
dtypes: int64(1), object(1)
memory usage: 31.3+ KB


,label,text,source
0,0,So there is no way for me to plug it in here i...,amazon
1,1,"Good case, Excellent value.",amazon
2,1,Great for the jawbone.,amazon
3,0,Tied to charger for conversations lasting more...,amazon
4,1,The mic is great.,amazon


(1000, 3)

preprocess the data (tokenize, stop word removal, lemmatization)

In [3]:
import nltk
#nltk.download('punkt')
#nltk.download('stopwords')
#nltk.download('wordnet')
#nltk.download('omw-1.4')
from nltk.tokenize import word_tokenize
import string
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

df_a['text'] = df_a['text'].str.lower()
df_a['tokens'] = df_a['text'].apply(lambda x : word_tokenize(x))
df_a['tokens'] = df_a['tokens'].apply(lambda x : [word for word in x if word.isalnum()]) 
stop_words = set(stopwords.words('english')+ list(string.punctuation))
df_a['tokens'] = df_a['tokens'].apply(lambda x : [w for w in x if not w in stop_words])
wordnet_lemmatizer = WordNetLemmatizer()
df_a['tokens'] = df_a['tokens'].apply(lambda x : [wordnet_lemmatizer.lemmatize(w) for w in x ])
df_a['Cleaned_Sentence'] = df_a['tokens'].apply(lambda x : ' '.join(x))

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(use_idf=True,  smooth_idf=False,ngram_range=(1,1))
tfidf_vectorizer.fit(df_a.Cleaned_Sentence)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=False,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

split the data to 20\% test and 80\%train and train a logistic regression model to claissfy the sentiment and convert to tfidf vectors

In [40]:
from sklearn import model_selection
from sklearn import preprocessing
x_train, x_test, y_train, y_test = model_selection.train_test_split(df_a.Cleaned_Sentence, df_a.label,test_size=0.2) 
 
encoder = preprocessing.LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.fit_transform(y_test)
x_train_tfidf = tfidf_vectorizer.transform(x_train)
x_test_tfidf = tfidf_vectorizer.transform(x_test)

from sklearn import linear_model
from sklearn import metrics

model=linear_model.LogisticRegression()
model.fit(x_train_tfidf, y_train)
predictions = model.predict(x_test_tfidf)
print(metrics.accuracy_score(predictions, y_test))


0.74


What are some example of sentiments claissified wrongly?

Below are some of the instances where sentiments can be wrongly classified : classified as 0 when it fits into 1 and vice versa.

In [50]:
#test data
count = 0
for input, prediction, label in zip(x_test, predictions, y_test):
    if prediction != label:
        print('"', input,'"', 'is classified as:', prediction, 'but should be classified as:', label)
        count= count +1
    if count>5:
        break;
    

" protects phone side " is classified as: 0 but should be classified as: 1
" say plantronics 510 maintains flawless connection cell static normal use " is classified as: 0 but should be classified as: 1
" total package " is classified as: 0 but should be classified as: 1
" usually like headband one lightweight mess hair " is classified as: 0 but should be classified as: 1
" got small adorable " is classified as: 0 but should be classified as: 1
" product exactly described " is classified as: 0 but should be classified as: 1


In [52]:
#train data
count = 0
for input, prediction, label in zip(x_train, predictions, y_train):
    if prediction != label:
        print('"', input,'"', 'is classified as:', prediction, 'but should be classified as:', label)
        count= count +1
    if count>5:
        break;

" aggravating " is classified as: 1 but should be classified as: 0
" wear everyday hold well " is classified as: 0 but should be classified as: 1
" battery last hour " is classified as: 1 but should be classified as: 0
" also problem phone reading memory card always turn " is classified as: 1 but should be classified as: 0
" good value work fine power via usb car wall outlet " is classified as: 0 but should be classified as: 1
" purchased family friend enjoy clarity ease use " is classified as: 0 but should be classified as: 1
